Hello! I am going to walk you through a program that enables Nao to take a picture of an object, save it, and then track it using its head.

First, we must import our ALProxy from Naoqi which will include all the APIs installed in our robot. Then, we will also import the libraries that we will use later. 

In [1]:
from naoqi import ALProxy
import time
import sys
import Image
import vision_definitions

Now, I want to create a Robot class and a constructor where I will initializes all the APIs I will use. 

In [2]:
class Robot0:
    def __init__(self, ip, port):
        self.ip = ip
        self.port = port
        
        #Initializes all the APIs I will use
        self.tts = ALProxy("ALTextToSpeech", ip, port)
        self.rbd = ALProxy("ALRedBallTracker", ip, port)
        self.visual = ALProxy("ALVideoDevice", ip, port)
        self.motion = ALProxy("ALMotion", ip, port)
        self.life = ALProxy("ALAutonomousLife", ip, port)
        self.posture = ALProxy("ALRobotPosture", ip, port)
        
        #Robot says "okay" after it initializes APIs
        #Disables robot's autonomous life
        self.tts.say("okay")
        self.life.setState("disabled")

Now, I define a method called save_photo() which will take a picture and save it to the directory. It will print out the image name once it's saved.

In [3]:
class Robot1(Robot0):
    def save_photo(self, img_name): #takes 30x40 photo and saves to .py file directory
        print ("Taking photo: " + img_name)
        vidClient = self.visual.subscribeCamera("python_client", 0, 2, 11, 5) #initialize cam with settings
        naoImage = self.visual.getImageRemote(vidClient) #taking photo
        time.sleep(0.5)
        self.visual.unsubscribe(vidClient) #unsubscribe
        imageWidth = naoImage[0]
        imageHeight = naoImage[1]
        array = naoImage[6]
        img = Image.frombytes("RGB", (imageWidth, imageHeight), array)
        img.save(img_name + ".jpeg", "JPEG")
        print ("Photo " + img_name + " saved!")

Now, that the image is saved I will define a method called set_neutral() that will set Nao's head angles to be in the neutral position.

In [4]:
class Robot2(Robot1):
    def set_neutral(self):
        self.motion.setStiffnesses("Head", 1.0)
        self.motion.setAngles("HeadPitch", 0, 0.1)
        self.motion.setAngles("HeadYaw", 0, 0.1)
        time.sleep(3.0)
        self.motion.setStiffnesses("Head", 0)

 I will also create a method called recognize_and_track() which will access the picture and recognize an specific object within it. 

In [5]:
class Robot(Robot2):
    def recognize_and_track(self, target_name, img_name):
        #Sets Nao's head to neutral, saves an image, then tracks the object
        #Says its found the object once it sees it
        #Sets head to neutral after tracker stops

        self.set_neutral()
        time.sleep(1.0)
        self.save_photo(img_name)
        self.motion.setStiffnesses("Head", 1.0)
        self.rbd.startTracker()
        print "Starting tracker"
        time.sleep(1.5)
        self.tts.say("Found " + target_name)
        time.sleep(15.0)
        print "Stopping tracker"
        self.rbd.stopTracker()
        self.motion.setStiffnesses("Head", 0)
        self.set_neutral()

Below, I will create a method called test() where I will call all the previous methods.  I will use Hedwig as an example and I will use a black tire/circle as the object to be recognized.

In [6]:
def test():
    hedwig = Robot("hedwig.brynmawr.edu", 9559)
    hedwig.recognize_and_track("circle", "circlePhoto")

Now, I will execute the code. 

In [7]:
test()

Taking photo: circlePhoto
Photo circlePhoto saved!
Starting tracker
Stopping tracker


Once executed, the program will print the following feedback:

In [5]:
%%HTML
<img src = "http://localhost:8888/files/test_code_feedback.png">

Also, here is the picture it took:

In [9]:
%%HTML
<IMG SRC = http://localhost:8888/files/circlePhoto.jpeg>